In [2]:
import numpy as np
import pygame
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


Cargar la base de datos de la simulación realizada

In [3]:
with open('ammonia_tank_mL_0.0e-6_kg_per_s.txt', 'r') as file:
    data = file.readlines()

Tank_area = np.pi * (0.201 ** 2) / 4 # d_i = 0.201 m
liquid_heights = []
times = []
BOG_rates = []
QV_values = []
BL_values = []

data = data[1:]
for element in data:
    cleaned_line = element.strip().split(',')
    # Add the values to the respective lists
    times.append(float(cleaned_line[0]))
    liquid_heights.append(float(cleaned_line[1]) / Tank_area)  # Convert m^3 to height in m
    BOG_rates.append(float(cleaned_line[2]))
    QV_values.append(float(cleaned_line[4]))
    BL_values.append(float(cleaned_line[5]))

In [4]:


# Inicializar pygame
pygame.init()

# Dimensiones de la ventana
WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Tanque esquemático")

# Colores
GRAY = (180, 180, 180)
WHITE = (255, 255, 255)
BLUE = (173, 216, 230)
PINK = (255, 220, 220)
BLACK = (0, 0, 0)

# Loop principal
running = True
while running:
    screen.fill(WHITE)
    
    # --- DIBUJO DEL TANQUE ---
    # Tanque externo
    tank_rect = pygame.Rect(120, 100, 160, 400)
    pygame.draw.rect(screen, GRAY, tank_rect, border_radius=80)
    
    # Fase líquida (parte inferior)
    liquid_rect = pygame.Rect(140, 360, 120, 120)
    # pygame.draw.rect(screen, BLUE, liquid_rect, border_radius=60)
    pygame.draw.rect(
    screen,
    BLUE,
    rect=liquid_rect,
    border_top_left_radius=0,
    border_top_right_radius=0,
    border_bottom_left_radius=60,
    border_bottom_right_radius=60
)
    
    # Fase vapor (parte superior)
    vapor_rect = pygame.Rect(140, 120, 120, 240)
    pygame.draw.rect(screen, PINK, vapor_rect, border_radius=60)
    pygame.draw.rect(
    screen,
    PINK,
    rect=vapor_rect,
    border_top_left_radius=60,
    border_top_right_radius=60,
    border_bottom_left_radius=0,
    border_bottom_right_radius=0
)
    
    # Contorno interior del tanque
    inner_rect = pygame.Rect(140, 120, 120, 360)
    pygame.draw.rect(screen, BLACK, inner_rect, 2, border_radius=60)
    
    # Línea de separación líquido-vapor
    pygame.draw.line(screen, BLACK, (140, 360), (260, 360), 2)
    
    # # --- Válvula superior ---
    # pygame.draw.polygon(screen, BLACK, [(200, 80), (190, 100), (210, 100)])  # triángulo inferior
    # pygame.draw.polygon(screen, BLACK, [(200, 60), (190, 80), (210, 80)])    # triángulo superior
    # pygame.draw.line(screen, BLACK, (200, 100), (200, 120), 3)  # tubo
    
    # # --- Bomba lateral ---
    # pygame.draw.line(screen, BLACK, (260, 420), (320, 420), 4)  # tubería horizontal
    # pygame.draw.circle(screen, GRAY, (340, 420), 20)             # cuerpo bomba
    # pygame.draw.circle(screen, BLACK, (340, 420), 20, 2)
    # pygame.draw.circle(screen, WHITE, (340, 420), 10)
    
    # Actualizar pantalla
    pygame.display.flip()
    
    # Salir del programa
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()
sys.exit()


KeyboardInterrupt: 

In [15]:
def meters_to_pixels(meters, scale=294.18):
    return int(meters * scale)

pygame.init()

# Ventana
WIDTH, HEIGHT = 400, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Animación de nivel de líquido")

# Colores
GRAY = (180, 180, 180)
WHITE = (255, 255, 255)
BLUE = (173, 216, 230)
PINK = (255, 220, 220)
BLACK = (0, 0, 0)

# Geometría del tanque
x_tank, y_tank, w_tank, h_tank = 120, 100, (120 + 60), (120 + 63)

# Datos de simulación (ejemplo)
time = times                 # 10 s simulados
height_liquid = liquid_heights  # ejemplo: sube asintóticamente

# Configurar reloj para controlar velocidad
clock = pygame.time.Clock()
fps = 30

# Escalar alturas a píxeles dentro del tanque
liquid_min = y_tank + h_tank - 20   # fondo
liquid_max = y_tank + 80            # parte superior del líquido

def get_liquid_y(h_fraction):
    # Convierte la fracción (0 a 1) en coordenada vertical
    return liquid_min - h_fraction * (liquid_min - liquid_max)

# Loop principal de animación
for i in range(len(time)):
    screen.fill(WHITE)

    h_bottom = (y_tank + h_tank - 20)
    h_surface_simulation = meters_to_pixels(height_liquid[i])

    # --- Tanque ---
    pygame.draw.rect(
        screen, GRAY, (x_tank, y_tank, w_tank, h_tank),
    )

    # Rectángulo interior del tanque
    pygame.draw.rect(
        screen, BLACK, (x_tank+20, y_tank+20, w_tank-40, h_tank-40),
        2, border_top_left_radius=60, border_top_right_radius=60
    )

    # --- Líquido ---
    # h_frac = (height_liquid[i] - min(height_liquid)) / (max(height_liquid) - min(height_liquid))
    y_liquid = h_bottom - h_surface_simulation
    height_px = h_surface_simulation
    liquid_rect = pygame.Rect(x_tank+20, y_liquid, w_tank-40, height_px)
    pygame.draw.rect(screen, BLUE, liquid_rect)

    # --- Vapor ---
    vapor_rect = pygame.Rect(x_tank+20, y_tank+20, w_tank-40, (y_liquid - (y_tank+20)))
    pygame.draw.rect(screen, PINK, vapor_rect)

    # --- Línea de separación ---
    pygame.draw.line(screen, BLACK, (x_tank+20, y_liquid), (x_tank+w_tank-20, y_liquid), 2)

    # Actualizar pantalla
    pygame.display.flip()
    clock.tick(fps)

    # Permitir salir del programa
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

pygame.quit()

KeyboardInterrupt: 